In [ ]:
knitr::opts_chunk$set(echo = TRUE)

################################################################################

This script downloads flow data from the pre-selected gages of interest. 
It also discovers stations with water quality data gathered by Vermont DEC 
corresponding to each of the streamflow gages and downloads that data. 
We have specifically tailored this approach to the Lake Champlain basin, but it is
flexible enough so that interested users can modify various lines to download
from the gages and sampling sites of interest to them. Most of this code will take
too long to run in a workshop capacity, but we have decided to include it here for 
completeness' sake. For the purposes of this workshop, we will simply read-in .csv files
with the requisite data

**Required inputs**

1) NWIS site numbers for sites of interest (data/lake_champlain_usgs_gages.csv)

**Outputs/Returns**

1) Raw dataframe of daily streamflow data for 18 watersheds in Lake Champlain

2) Raw water quality dataframe for total phosphorus and chloride concentration 
as measured in each of those 18 watersheds

################################################################################

# Housekeeping

In [ ]:
knitr::opts_chunk$set(echo = TRUE)


### Packages

In [ ]:

## Data mgmt
require(tidyverse)

## Data download
require(dataRetrieval)
require(nhdplusTools)
require(EPATADA)

## Misc.
require(here)


# Get metadata for USGS gage of interest

In [ ]:

#### First, retrieve the NWIS IDs of the gages we have selected
#### in the Lake Champlain basin
#### We've pulled these gage IDs from various pubs in the region 
#### (Underwood et al., 2017, WRR; Vaughn, 2019 LCBP Report)

lc_site_ids <- read_csv(here("input-data/lake_champlain_usgs_gages.csv")) %>%
  dplyr::select(river_basin, site_no)

#### Make sure the NWIS code is eight digits
#### .csv files often remove the leading zeros from ids

lc_site_ids <- lc_site_ids %>%
  mutate(site_no = as.character(sprintf("%08d", site_no)))

#### Get official site names and various other pieces of metadata

##### Specific to gage location and flow monitoring record

lc_gages_metadata <- whatNWISdata(siteNumber = lc_site_ids$site_no,
             parameterCd = "00060",
             service = "dv") 

##### Drainage area

lc_gages_metadata_da <- readNWISsite(lc_site_ids$site_no) %>%
  dplyr::select(site_no, drain_area_va)

##### And NHDPlus (Medium-res) COMID
##### Which is absolutely essential for working with the NWM
##### To do this we must first get the station IDs into the format needed
##### To query the NHD for the COMID related to each site location

lc_gages_nldi <- lc_site_ids %>%
  mutate(fsrc = "nwissite",
         fid = paste0("USGS-", site_no)) %>%
  mutate(comid = map2_dbl(fsrc, fid,
                      ~discover_nhdplus_id(nldi_feature = list(featureSource = .x, 
                                                featureID = .y)))) %>%
  dplyr::select(!c("river_basin", "fsrc"))

#### And clean up the metadata

lc_gages_metadata_clean <- inner_join(lc_site_ids, 
                                      lc_gages_nldi,
                                      by = "site_no") %>%
  inner_join(.,   lc_gages_metadata %>%
               dplyr::select(station_nm, site_no, 
                             dec_lat_va, dec_long_va, dec_coord_datum_cd,
                             begin_date, end_date),
             by = "site_no") %>%
  inner_join(., lc_gages_metadata_da,
             by = "site_no") %>%
  rename(drain_area_mi2 = drain_area_va) %>%
  mutate(drain_area_km2 = drain_area_mi2*2.58999) %>%
  dplyr::select(!drain_area_mi2) %>%
  relocate(drain_area_km2, .after = "station_nm") %>%
  rename(tributary = river_basin) %>%
  relocate(station_nm, .after = "tributary")




# Get metadata for water quality monitoring sites

In [ ]:

#### Now, read-in a list of the VTDEC tributary monitoring sites. 
#### To do so, we have to download data monitored by VTDEC 
#### for a subset of the Champlain Tribs water quality monitoring period (1991-present). 
#### Let's take a slice of 2012
#### Otherwise it would take waaaaaay too long to download
#### 2012, according to VTDEC's webpage, should be a period when monitoring in all tribs
#### is active

#### Note that this returns every sample sampled by VTDEC in this slice of 2012

wq_data_profile <- EPATADA::TADA_DataRetrieval(
                           organization = "1VTDECWQ",
                           startDate = "2012-03-01",
                           endDate = "2012-09-30",
                           applyautoclean = FALSE)

#### Map locations of samples and inspect to make sure they fall in both
#### VT & NY and include all the tribs we want

EPATADA::TADA_OverviewMap(wq_data_profile %>%
                         rename(TADA.LatitudeMeasure = ActivityLocation.LatitudeMeasure,
                                TADA.LongitudeMeasure = ActivityLocation.LongitudeMeasure,
                                TADA.CharacteristicName = CharacteristicName) %>%
                         mutate(TADA.LatitudeMeasure = as.numeric(TADA.LatitudeMeasure),
                                TADA.LongitudeMeasure =as.numeric(TADA.LongitudeMeasure)))

#### Select river monitoring sites that fall along the main stem of each tributary
#### Then, slice by the number of samples in our sample monitoring period
#### We will select the site with the most samples, which we know will correspond
#### to the main monitoring site for water quality for each of the tribs
#### (which is the site we want)
#### We will then use these site ids to download all the water quality data for the monitoring period

site_ids_wq <- wq_data_profile %>%
  filter(MonitoringLocationName %in% lc_gages_metadata_clean$tributary) %>%
  dplyr::select(MonitoringLocationIdentifier, MonitoringLocationName, ResultIdentifier) %>%
  dplyr::group_by(MonitoringLocationIdentifier,MonitoringLocationName) %>%
  summarise(sample_count = length(unique(ResultIdentifier))) %>%
  dplyr::ungroup() %>%
  dplyr::group_by(MonitoringLocationName) %>%
  slice_max(sample_count)

#### Now plot again to make sure that we selected the right ones

EPATADA::TADA_OverviewMap(wq_data_profile %>%
                         rename(TADA.LatitudeMeasure = ActivityLocation.LatitudeMeasure,
                                TADA.LongitudeMeasure = ActivityLocation.LongitudeMeasure,
                                TADA.CharacteristicName = CharacteristicName) %>%
                         mutate(TADA.LatitudeMeasure = as.numeric(TADA.LatitudeMeasure),
                                TADA.LongitudeMeasure =as.numeric(TADA.LongitudeMeasure)) %>%
                         filter(MonitoringLocationIdentifier %in% site_ids_wq$MonitoringLocationIdentifier))

#### And finally join the stream gage metadata file so that we have all
#### station identifiers in one location

lc_sites_metadata_all <- inner_join(lc_gages_metadata_clean,
           site_ids_wq %>%
             dplyr::select(MonitoringLocationIdentifier, MonitoringLocationName) %>%
             rename(wq_site_id = MonitoringLocationIdentifier,
                    tributary = MonitoringLocationName),
           by = "tributary") %>%
  relocate(wq_site_id, .after = fid)


################################

#### Remove extraneous variables

rm(wq_data_profile)

################################



# Download Data

### Streamflow Data

In [ ]:

#### Downloads USGS flow data 
#### We want to daily values
#### so let's download them

flow_data <- dataRetrieval::readNWISdv(siteNumbers = lc_sites_metadata_all$site_no, 
                          parameterCd = "00060",
                          startDate = "1990-01-01",
                          endDate = "2023-12-31") %>%
  renameNWISColumns() %>%
  addWaterYear() %>%
  dplyr::filter(!str_detect(Flow_cd , "P"))

write_csv(flow_data, here("output-data/raw_streamflow_data.csv"))

### Water quality data

In [ ]:

#### Download water quality data from each of the eighteen tribs
#### This require looping over each station to avoid breaking the downloader

tribs_wq <- list()

for(i in 1:length(lc_sites_metadata_all$wq_site_id)) {
  
  cat(crayon::cyan("Reading", lc_sites_metadata_all$tributary[i], "\n"))
  
  wq_by_site <- TADA_DataRetrieval(siteid = lc_sites_metadata_all$wq_site_id[i],
                           startDate = "1990-01-01",
                           endDate = "2023-12-31",
                           applyautoclean = FALSE)
  
  
  tribs_wq[[i]] <- wq_by_site
  
}



lc_tribs_wq_all <- bind_rows(tribs_wq)




# Clean Data

### Clean the USGS streamflow data

In [ ]:

#### Join to tables with site names
#### Transform flow data to m3/s 
#### Take 0 flow days and make them instead a very small number (10^-4 cfs)
#### And finally normalize by watershed area

flow_data_clean <- flow_data %>%
  dplyr::select(site_no, Date, Flow, waterYear) %>%
  rename(date = Date,
         discharge_cfs = Flow) %>%
  mutate(discharge_cfs = ifelse(discharge_cfs == 0, 1E-4, discharge_cfs)) %>%
  mutate(discharge_cms = discharge_cfs*0.0283168) %>%
  inner_join(lc_gages_metadata_clean %>%
               dplyr::select(tributary, site_no),
             .,
             by = "site_no") %>%
  dplyr::ungroup() 



### Clean VTDEC water quality monitoring data

In [ ]:

#### Mostly, select the constituents we want, rename, and join columns to get 
#### a more workable format
#### The most important thing we are doing here is joining the constituent with 
#### its measured fraction to make one variable 
#### so we can distinguish between, say, total and dissolved phosphorus
#### We are also dropping missing data, making sure concentrations are numeric
#### and transforming dates to datetime format
#### Also, if there are multiple observations on the same date at the same site,
#### we take the mean to get one observation per site per date
#### We then want to add in the missing data from Little Otter Creek that we 
#### had to download manually from VTDEC's website
#### We then want to make sure Chloride, which for some reason is sometimes listed 
#### as total OR dissolved, is all under one moniker
#### (Because there is no difference between the two)
#### And finally select only Chloride and Total Phosphorus

lc_tribs_wq_all_clean <- lc_tribs_wq_all %>%
  dplyr::select(where(~!all(is.na(.x)))) %>% ### Drop the columns that have all NAs
  dplyr::select( 
                MonitoringLocationName, MonitoringLocationIdentifier,
                ActivityLocation.LatitudeMeasure, ActivityLocation.LongitudeMeasure,
                CharacteristicName, ResultSampleFractionText,
                ActivityStartDate, ActivityStartTime.Time,
                ResultMeasureValue, ResultMeasure.MeasureUnitCode) %>%
  rename(tributary = MonitoringLocationName,
         storet_site_id = MonitoringLocationIdentifier, 
         latitude = ActivityLocation.LatitudeMeasure,
         longitude = ActivityLocation.LongitudeMeasure,
         constituent = CharacteristicName,
         fraction = ResultSampleFractionText,
         date = ActivityStartDate,
         time = ActivityStartTime.Time,
         conc = ResultMeasureValue,
         units = ResultMeasure.MeasureUnitCode) %>%
  mutate(constituent = ifelse(!is.na(fraction), 
                              paste0(constituent, "_", fraction),
                              constituent)) %>%
  dplyr::select(!fraction) %>%
  as_tibble() %>%
  mutate(conc = as.numeric(conc)) %>%
  drop_na(conc) %>%
  mutate(date = as_date(date)) %>%
  dplyr::group_by(tributary, date, constituent) %>%
  mutate(conc = mean(conc)) %>%
  dplyr::slice(1) %>%
  dplyr::ungroup() %>%
  dplyr::select(tributary, date, constituent, conc) %>%
  mutate(constituent = ifelse(str_detect(constituent, "Chloride"), "Chloride", constituent)) %>%
  dplyr::filter(constituent %in% c("Phosphorus_Total", "Chloride"))
  

